# Dataset-Generator.

In [1]:
import os
from datagenerator import DataGenerator
import pickle
import random
import datetime

Using TensorFlow backend.


# Setting the parameters.

In [2]:
if os.path.exists("datasetpath.txt"):
    dataset_path = open("datasetpath.txt", "r").read().replace("\n", "")
else:
    dataset_path = "../data"

# For creating voxelgrids.
dataset_parameters_voxelgrids = {}
dataset_parameters_voxelgrids["input_type"] = "voxelgrid"
dataset_parameters_voxelgrids["output_targets"] = ["height", "weight"]    
dataset_parameters_voxelgrids["random_seed"] = 666
dataset_parameters_voxelgrids["voxelgrid_target_shape"] = (32, 32, 32)
dataset_parameters_voxelgrids["voxel_size_meters"] = 0.1
dataset_parameters_voxelgrids["voxelgrid_random_rotation"] = True
dataset_parameters_voxelgrids["dataset_size_train"] = 6000
dataset_parameters_voxelgrids["dataset_size_test"] = 1000

# For creating pointclouds.
dataset_parameters_pointclouds = {}
dataset_parameters_pointclouds["input_type"] = "pointcloud"
dataset_parameters_pointclouds["output_targets"] = ["height", "weight"]    
dataset_parameters_pointclouds["random_seed"] = 666
dataset_parameters_pointclouds["pointcloud_target_size"] = 30000
dataset_parameters_pointclouds["pointcloud_random_rotation"] = True
dataset_parameters_pointclouds["dataset_size_train"] = 3000
dataset_parameters_pointclouds["dataset_size_test"] = 500

# Creating the data-generator.
Makes use of the parameters.

In [3]:
def create_datagenerator(dataset_parameters):
    print("Creating data-generator...")
    datagenerator = DataGenerator(
        dataset_path=dataset_path, 
        input_type=dataset_parameters["input_type"], 
        output_targets=dataset_parameters["output_targets"],
        voxelgrid_target_shape=dataset_parameters.get("voxelgrid_target_shape", None),
        voxel_size_meters=dataset_parameters.get("voxel_size_meters", None),
        voxelgrid_random_rotation=dataset_parameters.get("voxelgrid_random_rotation", None),
        pointcloud_target_size=dataset_parameters.get("pointcloud_target_size", None),
        pointcloud_random_rotation=dataset_parameters.get("pointcloud_random_rotation", None)
    )
    datagenerator.print_statistics()
    return datagenerator

# Analysis.

In [4]:
def analyze(datagenerator):
    datagenerator.analyze_files()
    datagenerator.analyze_targets()
    datagenerator.analyze_pointclouds()
    datagenerator.analyze_voxelgrids()

# Do the train-test-split and generate.

In [5]:
def split_and_generate(datagenerator, dataset_parameters):

    # Do the split.
    random.seed(dataset_parameters["random_seed"])
    qrcodes_shuffle = datagenerator.qrcodes[:]
    random.shuffle(qrcodes_shuffle)
    split_index = int(0.8 * len(qrcodes_shuffle))
    qrcodes_train = sorted(qrcodes_shuffle[:split_index])
    qrcodes_test = sorted(qrcodes_shuffle[split_index:])
    del qrcodes_shuffle
    print("")

    print("QR-Codes for training:", " ".join(qrcodes_train))
    print("")
    print("QR-Codes for testing:", " ".join(qrcodes_test))
    print("")

    print("Generating training data...")
    dataset_train = next(datagenerator.generate(size=dataset_parameters["dataset_size_train"], qrcodes_to_use=qrcodes_train, verbose=True))

    print("Generating testing data...")
    dataset_test = next(datagenerator.generate(size=dataset_parameters["dataset_size_test"], qrcodes_to_use=qrcodes_test, verbose=True))

    print("Done.")
    return dataset_train, dataset_test

# Method for saving dataset.

In [6]:
def save_dataset(dataset_train, dataset_test, dataset_parameters):
    print("Saving dataset...")
    datetime_string = datetime.datetime.now().strftime("%Y%m%d-%H%M")
    dataset_name = datetime_string + "-" + dataset_parameters["input_type"] + "-dataset.p"
    pickle.dump((dataset_train, dataset_test, dataset_parameters), open(dataset_name, "wb"))
    print("Saved " + dataset_name)

# Generate with parameters.

In [7]:
dataset_parameters_to_use = []
dataset_parameters_to_use.append(dataset_parameters_pointclouds)
dataset_parameters_to_use.append(dataset_parameters_voxelgrids)

for dataset_parameters in dataset_parameters_to_use:
    
    datagenerator = create_datagenerator(dataset_parameters)
    #analyze(datagenerator)
    
    dataset_train, dataset_test = split_and_generate(datagenerator, dataset_parameters)
    
    save_dataset(dataset_train, dataset_test, dataset_parameters)
    

Creating data-generator...
/Users/tristanbehrens/Datasets/welthungerhilfe/20180727
QR-Code SAM-GOV-025 has 1 different manual measurements
   Target [67, 6.7] with 160 JPGs and 51 PCDs.
QR-Code SAM-GOV-003 has 2 different manual measurements
   Target [92.7, 9.7] with 0 JPGs and 0 PCDs.
   Target [75.3, 7.1] with 0 JPGs and 0 PCDs.
QR-Code SAM-SNG-073 has 3 different manual measurements
   Target [82, 8.8] with 122 JPGs and 41 PCDs.
   Target [82, 8.8] with 136 JPGs and 40 PCDs.
   Target [82, 8.5] with 0 JPGs and 0 PCDs.
QR-Code SAM-GOV-014 has 1 different manual measurements
   Target [91.1, 11.1] with 279 JPGs and 77 PCDs.
QR-Code SAM-GOV-087 has 1 different manual measurements
   Target [85.7, 10.2] with 225 JPGs and 60 PCDs.
QR-Code SAM-GOV-004 has 1 different manual measurements
   Target [76.2, 8.3] with 235 JPGs and 71 PCDs.
QR-Code SAM-GOV-026 has 1 different manual measurements
   Target [80.1, 9.2] with 145 JPGs and 47 PCDs.
QR-Code SAM-GOV-068 has 1 different manual measure

  4% (147 of 3000) |#                    | Elapsed Time: 0:00:50 ETA:   0:15:03

Wrong number of columns at line 26769


  6% (203 of 3000) |#                    | Elapsed Time: 0:01:10 ETA:   0:16:46

Wrong number of columns at line 26558


...

Wrong number of columns at line 25900


 99% (991 of 1000) |#################### | Elapsed Time: 0:05:16 ETA:   0:00:02

Wrong number of columns at line 25977


100% (1000 of 1000) |####################| Elapsed Time: 0:05:19 Time:  0:05:19


Done.
Saved 20180731-1325-voxelgrid-dataset.p
